In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula_middle_padding import Tabula 
import pandas as pd
import torch

In [2]:
data = pd.read_csv("Real_Datasets/customer_churn.csv")

In [3]:
#categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
model = Tabula(llm='distilgpt2', experiment_dir = "customer-churn_training", batch_size=32, epochs=128)

## In following block, it is important to indicate "conditional_col = data.columns[0]". Otherwise, the generation will use target column as conditional column, which will result in error!

In [4]:
model.fit(data, conditional_col = data.columns[0])

/home/svenlag/tabula_test/Tabula/tabula_middle_padding/tabula.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.198000
1000,1.520700
1500,1.172500
2000,0.864000
2500,0.642600
3000,0.521800
3500,0.464600
4000,0.436800
4500,0.420500
5000,0.409100


In [5]:
import torch
torch.save(model.model.state_dict(), "customer-churn_training/model_128epoch_middle_padding.pt")

In [5]:
torch.save(model, "customer-churn_training/model_128epoch_middle_padding_FULL.pt")

In [3]:
# Specify weights_only=False to allow full unpickling
model = torch.load(
    "customer-churn_training/model_128epoch_middle_padding_FULL.pt",
    map_location=torch.device("cuda"),      # or "cpu" as needed
    weights_only=False
)
model.model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
for i in range(1,31):
    synthetic_data = model.sample(n_samples=3150)
    synthetic_data.to_csv(f"customer-churn_sd-sets/customer-churn_128epoch_middle_padding_{i}.csv", index=False)

  0%|          | 0/3150 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
3200it [01:05, 48.90it/s]                          
3200it [01:06, 48.11it/s]                          
3200it [01:09, 46.22it/s]                          
3200it [01:09, 46.06it/s]                          
3200it [01:09, 45.92it/s]                          
3200it [01:09, 45.71it/s]                          
3200it [01:11, 44.58it/s]                          
3199it [01:12, 43.87it/s]                          
3200it [01:13, 43.65it/s]                          
3200it [01:14, 42.98it/s]                          
3200it [01:15, 42.40it/s]                          
3200it [01:16, 42.02it/s]                          
3200it [01:16, 42.09it/s]                          
3200it [01:16, 41.85it/s]                          
320

In [6]:
synthetic_data = model.sample(n_samples=3150)
synthetic_data.to_csv("customer-churn_128epoch_middle_padding.csv", index=False)

  0%|          | 0/3150 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
3200it [01:52, 28.40it/s]                          


In [8]:
from sdmetrics.reports.single_table import QualityReport

synthetic_data = synthetic_data[data.columns.to_list()]

metadata = {
    "columns": {
        "Call_Failure": {"sdtype": "numerical"},
        "Complains": {"sdtype": "numerical"},
        "Subscription_Length": {"sdtype": "numerical"},
        "Charge_Amount": {"sdtype": "numerical"},
        "Seconds_of_Use": {"sdtype": "numerical"},
        "Frequency_of_use": {"sdtype": "numerical"},
        "Frequency_of_SMS": {"sdtype": "numerical"},
        "Distinct_Called_Numbers": {"sdtype": "numerical"},
        "Age_Group": {"sdtype": "numerical"},
        "Tariff_Plan": {"sdtype": "numerical"},
        "Status": {"sdtype": "numerical"},
        "Age": {"sdtype": "numerical"},
        "Customer_Value": {"sdtype": "numerical"},
        "Churn": {"sdtype": "numerical"}
    }
}

report = QualityReport()
report.generate(data, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 14/14 [00:00<00:00, 425.04it/s]|
Column Shapes Score: 96.88%



/home/svenlag/tabula_test/tabula_env/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning:

ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.



(2/2) Evaluating Column Pair Trends: |██████████| 91/91 [00:00<00:00, 309.40it/s]|
Column Pair Trends Score: 98.5%

Overall Score (Average): 97.69%

